In [ ]:
import concurrent.futures
from functools import partial
from glob import glob
import os

import matplotlib.pyplot as plt
import numpy as np
from obspy import Stream
import pygmt
from pysep.utils.io import read_sem
from scipy.signal import hilbert as analytic

In [ ]:
write = False

In [ ]:
map = 'Brown'
database_type = 'SymGroups'
#database_type = 't20'
#database_type = 't2_5'

In [ ]:
# choose only two model types
if database_type == 'SymGroups':
    #models = ['ISO', 'XISO']
    models = ['MONO', 'TRIV']
elif database_type == 't20':
    models = ['t00', 't20']
    #models = ['t80', 't100']
elif database_type == 't2_5':
    models = ['t00', 't20']

misfit_functions = ['waveform', 'envelope', 'instantaneous_phase', 'traveltime']
#misfit_functions = ['envelope']

misfit_components = ['Z', 'Y', 'X']
#misfit_components = ['Z']

time_length_s = 40 # corresponds to -1s to 39s of data

database_loc = f'/scratch/agupta7/specfem/rectangular_grid/{map}_{database_type}' 
source_file = f'{database_loc}/{models[0]}/OUTPUT_FILES/CMTSOLUTION'
stations_file = f'{database_loc}/{models[0]}/OUTPUT_FILES/STATIONS_FILTERED'

In [ ]:
# misfit function definitions
# after https://github.com/adjtomo/seisflows (commit 6afdd56)

def waveform(st_syn, st_obs):
    misfit = 0
    nt = st_obs[0].stats.npts 
    dt = st_obs[0].stats.delta
    for component in misfit_components:
        syn = st_syn.select(component=component)[0].data
        obs = st_obs.select(component=component)[0].data
        wrsd = syn - obs
        misfit += np.sqrt(np.sum(wrsd * wrsd * dt))
    return misfit

def envelope(st_syn, st_obs):
    misfit = 0
    nt = st_obs[0].stats.npts 
    dt = st_obs[0].stats.delta
    for component in misfit_components:
        syn = st_syn.select(component=component)[0].data
        obs = st_obs.select(component=component)[0].data
        env_syn = abs(analytic(syn))
        env_obs = abs(analytic(obs))
        env_rsd = env_syn - env_obs
        misfit += np.sqrt(np.sum(env_rsd * env_rsd * dt))
    return misfit

def instantaneous_phase(st_syn, st_obs):
    misfit = 0
    nt = st_obs[0].stats.npts 
    dt = st_obs[0].stats.delta
    for component in misfit_components:
        syn = st_syn.select(component=component)[0].data
        r = np.real(analytic(syn))
        i = np.imag(analytic(syn))
        phi_syn = np.arctan2(i, r)
        obs = st_obs.select(component=component)[0].data
        r = np.real(analytic(obs))
        i = np.imag(analytic(obs))
        phi_obs = np.arctan2(i, r)
        phi_rsd = phi_syn - phi_obs
        misfit += np.sqrt(np.sum(phi_rsd * phi_rsd * dt))
    return misfit

def traveltime(st_syn, st_obs):
    misfit = 0
    nt = st_obs[0].stats.npts 
    dt = st_obs[0].stats.delta
    for component in misfit_components:
        syn = st_syn.select(component=component)[0].data
        obs = st_obs.select(component=component)[0].data
        cc = abs(np.convolve(obs, np.flipud(syn)))
        misfit += (np.argmax(cc) - nt + 1) * dt
    return misfit

In [ ]:
st_list = []
workers = os.cpu_count()
read_sem_new = partial(read_sem, source=source_file, stations=stations_file)

for model in models:   
    print(f'loading data for model: {model}')
    
    fids = []
    for fid in glob(f'{database_loc}/{model}/OUTPUT_FILES/seismograms/*R*'):
        fids.append(fid)
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
        data = list(executor.map(read_sem_new, fids))
    
    st = Stream()
    for st_data in data:
        st += st_data
    st_list.append(st)

In [ ]:
for st in st_list:
    for tr in st:
        t1 = tr.stats.starttime
        t2 = t1 + time_length_s
        tr.trim(t1,t2)
        
st1_all = st_list[0].copy().differentiate()
st2_all = st_list[1].copy().differentiate()

In [ ]:
stations_info = np.genfromtxt(stations_file, dtype=str)

stations = stations_info[:,0]
latitudes = np.array([float(i) for i in stations_info[:,2]])
longitudes = np.array([float(i) for i in stations_info[:,3]])

mask = np.char.startswith(stations, 'C')

stations = stations[~mask]
latitudes = latitudes[~mask]
longitudes = longitudes[~mask]

In [ ]:
misfit = np.zeros((len(misfit_functions),len(stations)))

In [ ]:
# compute misfit

st1s = [st1_all.select(station=station) for station in stations]
st2s = [st2_all.select(station=station) for station in stations]

for i, misfit_function in enumerate(misfit_functions):
    
    if misfit_function=='waveform':
        with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
            data = list(executor.map(waveform, st2s, st1s))
        misfit[i,:] = np.array(data)
    elif misfit_function=='envelope':
        with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
            data = list(executor.map(envelope, st2s, st1s))
        misfit[i,:] = np.array(data)
    elif misfit_function=='instantaneous_phase':
        with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
            data = list(executor.map(instantaneous_phase, st2s, st1s))
        misfit[i,:] = np.array(data)
    elif misfit_function=='traveltime':
        with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
            data = list(executor.map(traveltime, st2s, st1s))
        misfit[i,:] = np.array(data)
                
if write:
    for i, misfit_function in enumerate(misfit_functions):
        np.save(f'{misfit_function}', [misfit[i,:], latitudes, longitudes])

In [ ]:
# misfit distribution across stations

# highlight stations with low and high misfit
stations_selected = ['R33-25', 'R18-17']
    
x = longitudes/1000
y = latitudes/1000

for i, misfit_function in enumerate(misfit_functions):

    # uncomment the following line of code to load pre-saved misfit values 
    # misfit[i,:], latitudes, longitudes = np.load(f""
    # f"../Data/Brown2016/misfit_{misfit_function}.npy")

    fig = pygmt.Figure()
    fig.basemap(region=[-156, 156, -156, 156], projection="X10c/10c", 
                frame=True)
    pygmt.makecpt(cmap="jet", 
                  series=[1.01*np.min(misfit[i,:]), 1.01*np.max(misfit[i,:])])
    fig.plot(x=x, y=y, style='t0.1c', fill=misfit[i,:], cmap=True)
    
    # highlight stations with low and high misfit
    for station in stations_selected:
        st_select = st1_all.select(station=station)
        x1 = st_select[0].stats.sac['stlo'] / 1000
        y1 = st_select[0].stats.sac['stla'] / 1000
        fig.plot(x=x1, y=y1, style='c0.2c', pen='0.75p,black')
    
    fig.colorbar(frame=[f"x+l{misfit_function} misfit", "y+l "])
    if write: fig.savefig(f'variability_scatter_{models[0]}_vs_{models[1]}_'
                          f'{misfit_function}_misfit.png', bbox_inches='tight')
    fig.show()

In [ ]:
indices = np.argsort(misfit[0,:])
sorted_stations = stations[indices]

# low misfit stations starting with the lowest
print(f"low misfit stations\n{sorted_stations[0:100]}\n")

# high misfit stations starting with the highest
print(f"high misfit stations\n{sorted_stations[-100:][::-1]}")

In [ ]:
# plot seismograms for a station

station = "R33-25"
#station = "R18-17"

plot_components = ['Z', 'Y', 'X']

vp = np.sqrt(118.3333E9/3378)
vs = np.sqrt(41.4333E9/3378)

st1 = st1_all.select(station=station)
st2 = st2_all.select(station=station)

x1 = st1[0].stats.sac['stlo']
y1 = st1[0].stats.sac['stla']
x2 = st1[0].stats.sac['evlo']
y2 = st1[0].stats.sac['evla']
evdp = st1[0].stats.sac['evdp'] * 1000

source_station_distance = np.sqrt( (y2-y1)**2 + (x2-x1)**2 + evdp**2 )

tp = source_station_distance / vp
ts = source_station_distance / vs

fig, axs = plt.subplots(3,1,figsize=(10,20)) 
max_velocity = 0
for i, component in enumerate(plot_components):
    tr1 = st1.select(component=component)[0]
    tr2 = st2.select(component=component)[0]
    axs[i].plot(tr1.times()+tr1.stats.sac['b'], tr1.data, c='k', 
                label=f'{models[0]}')
    axs[i].plot(tr2.times()+tr2.stats.sac['b'], tr2.data, c='r', 
                label=f'{models[1]}') 
    axs[i].axvline(x=tp, color='b', linestyle='--')
    axs[i].axvline(x=ts, color='b', linestyle='--')
    axs[i].set_title(f'component = {component}')
    axs[i].set_xlabel('time (s)')
    axs[i].set_ylabel('velocity (m/s)')
    axs[i].legend()
    max_velocity = max([max_velocity, max(abs(tr1.data)), max(abs(tr2.data))])
    
for i, _ in enumerate(plot_components):
    axs[i].set_ylim([-1.1 * max_velocity, 1.1 * max_velocity])
    
fig.suptitle(f'{station}')
plt.show()